# RTM Simulation H: Quantum-Confined Regime (α ≈ 3.5)## Multiscale Temporal Relativity — Computational Validation**Theoretical prediction:** α = d + ½ = 3.5 (Loop Quantum Gravity)**Model:** Pure 3D cubic lattice + quantum harmonic confinement potential.No long-range links — transport is purely diffusive within a confining geometrywhere boundary nodes experience self-loops scaling as β·L^γ.**Key result:** α = 3.4907 ± 0.0677, 95% CI includes 3.5 ✓

In [ ]:
import numpy as npimport pandas as pdimport matplotlib.pyplot as plt%matplotlib inline# Load resultsresults = pd.read_csv("quantum_confined_results.csv")walks = pd.read_csv("quantum_confined_walks.csv")fit_summary = pd.read_csv("quantum_confined_fit_summary.csv")print("Results loaded successfully.")print(f"\nLattice sizes: {sorted(results['L'].unique())}")print(f"Total walks: {len(walks):,}")print(f"\nFit summary:")print(fit_summary[['alpha','alpha_se','R2','bs_ci_lo','bs_ci_hi','delta_alpha']].to_string(index=False))

## Data Summary

In [ ]:
# Display full results tablecols = ['L', 'N', 'T_mean', 'T_std', 'T_ci_low', 'T_ci_high',        'completed', 'completion_pct', 'boundary_pct', 'self_loops_per_boundary']print(results[cols].to_string(index=False, float_format='{:,.1f}'.format))

## Figure 1: Log-Log Power Law Fit

In [ ]:
L = results['L'].values.astype(float)T = results['T_mean'].values.astype(float)Tstd = results['T_std'].values.astype(float)# FitlogL = np.log10(L); logT = np.log10(T)A = np.vstack([logL, np.ones(len(logL))]).Talpha, intercept = np.linalg.lstsq(A, logT, rcond=None)[0]C = 10**interceptfig, ax = plt.subplots(figsize=(10, 7))ax.errorbar(L, T, yerr=Tstd, fmt='o', color='#7C3AED', ms=10,            capsize=5, capthick=2, lw=2, label='Simulation',            markeredgecolor='white', markeredgewidth=1.5)Lf = np.linspace(L.min()*0.85, L.max()*1.15, 300)ax.plot(Lf, C*Lf**alpha, '-', color='#7C3AED', lw=2.5,        label=f'Fit: α = {alpha:.4f}')C35 = T[0]/(L[0]**3.5)ax.plot(Lf, C35*Lf**3.5, '--', color='#059669', lw=2, alpha=0.8,        label='Target α = 3.5')C30 = T[0]/(L[0]**3.0)ax.plot(Lf, C30*Lf**3.0, ':', color='#DC2626', lw=1.5, alpha=0.6,        label='Holographic α = 3.0')ax.set_xscale('log'); ax.set_yscale('log')ax.set_xlabel('L', fontsize=14); ax.set_ylabel('MFPT T', fontsize=14)ax.set_title(f'Quantum-Confined: α = {alpha:.4f}', fontsize=15, fontweight='bold')ax.legend(fontsize=11); ax.grid(True, alpha=0.3, which='both')plt.tight_layout(); plt.show()

## Figure 2: Finite-Size Convergence

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))running_a = []for k in range(3, len(L)+1):    lL = np.log10(L[:k]); lT = np.log10(T[:k])    Ak = np.vstack([lL, np.ones(k)]).T    ak = np.linalg.lstsq(Ak, lT, rcond=None)[0][0]    running_a.append(ak)ks = np.arange(3, len(L)+1)ax.plot(ks, running_a, 's-', color='#7C3AED', ms=9, lw=2,        markeredgecolor='white', markeredgewidth=1.5)ax.axhline(3.5, color='#059669', ls='--', lw=2.5, label='Theory α=3.5')ax.axhline(3.0, color='#DC2626', ls=':', lw=1.5, alpha=0.6, label='Holographic α=3.0')ax.fill_between(ks, 3.3, 3.7, alpha=0.08, color='#059669')ax.set_xticks(ks)ax.set_xticklabels([f'{k}\n(L≤{int(L[k-1])})' for k in ks])ax.set_ylabel('Fitted α', fontsize=13)ax.set_title('Convergence toward α = 3.5', fontsize=14, fontweight='bold')ax.legend(fontsize=11); ax.grid(True, alpha=0.3)plt.tight_layout(); plt.show()

## Figure 3: Confinement MechanismThe quantum confinement works through two competing effects:- **Boundary fraction** decreases as L grows (surface-to-volume ratio ∝ L⁻¹)- **Self-loops per boundary node** increase as L grows (∝ β·L^γ)- Net effect: O(1) correction → constant additive shift to the exponent

In [ ]:
fig, ax1 = plt.subplots(figsize=(10, 6))ax2 = ax1.twinx()ax1.bar(range(len(L)), results['boundary_pct'], color='#7C3AED',        alpha=0.5, label='Boundary %')ax2.plot(range(len(L)), results['self_loops_per_boundary'], 's-',         color='#DC2626', ms=8, lw=2, label='Self-loops/boundary node')ax1.set_xticks(range(len(L)))ax1.set_xticklabels([f'L={int(l)}' for l in L])ax1.set_ylabel('Boundary Nodes (%)', color='#7C3AED', fontsize=13)ax2.set_ylabel('Self-loops / Boundary Node', color='#DC2626', fontsize=13)ax1.set_title('Confinement Mechanism', fontsize=14, fontweight='bold')ax1.legend(loc='center left'); ax2.legend(loc='center right')ax1.grid(True, alpha=0.3)plt.tight_layout(); plt.show()

## RTM Scaling Spectrum: All 7 RegimesWith this simulation, **all 7 RTM regimes have been computationally validated**,from ballistic (α=1) through quantum-confined (α≈3.5).

In [ ]:
regimes = [    ('Ballistic', 1.0, 1.00), ('Diffusive', 2.0, 1.97),    ('Sierpiński', 2.32, 2.32), ('Vascular', 2.5, 2.39),    ('Hierarchical', 2.6, 2.67), ('Holographic', 3.0, 2.95),    ('Quantum-\nConfined', 3.5, alpha),]fig, ax = plt.subplots(figsize=(12, 6))x = np.arange(len(regimes))for i, (n, th, m) in enumerate(regimes):    c = '#7C3AED' if i == len(regimes)-1 else '#2563EB'    ax.bar(i-0.15, th, 0.3, color='#059669', alpha=0.5)    ax.bar(i+0.15, m, 0.3, color=c, alpha=0.85)ax.set_xticks(x); ax.set_xticklabels([r[0] for r in regimes])ax.set_ylabel('α', fontsize=14); ax.set_ylim(0, 4.5)ax.set_title('Complete RTM Scaling Spectrum', fontsize=14, fontweight='bold')ax.legend(['Theory', 'Measured'], fontsize=11, loc='upper left')ax.grid(True, alpha=0.3, axis='y')plt.tight_layout(); plt.show()print(f"\nQuantum-Confined result: α = {alpha:.4f}")print(f"Bootstrap 95% CI: [{fit_summary['bs_ci_lo'].iloc[0]:.4f}, "      f"{fit_summary['bs_ci_hi'].iloc[0]:.4f}]")print(f"Theory target: 3.5")print(f"Status: {'VALIDATED ✓' if fit_summary['bs_ci_lo'].iloc[0] <= 3.5 <= fit_summary['bs_ci_hi'].iloc[0] else 'Needs validation'}")